In [ ]:
import numpy as np
import jupyter_black
jupyter_black.load(lab=False)


In [ ]:
with open("day14.txt", "r") as f:
    data = f.read()

In [ ]:
def get_edge(rocks, f=min, coord=0):
    return f([f(path[:, coord]) for path in rocks])


def map_out_rocks(rocks, grid):

    for path in rocks:
        for i in range(len(path) - 1):
            d = path[i + 1] - path[i]
            if d[0] != 0:
                for j in range(0, d[0], np.sign(d[0])):
                    point = path[i] + np.array([j, 0])
                    grid[tuple(point)] = "#"
            if d[1] != 0:
                for j in range(0, d[1], np.sign(d[1])):
                    point = path[i] + np.array([0, j])
                    grid[tuple(point)] = "#"

            grid[tuple(path[i + 1])] = "#"


def viz_grid(grid):
    for line in grid.T:
        print("".join(line))

In [ ]:
class AbyssError(Exception):
    pass


class NozzleProblem(Exception):
    pass


def sand_drop(grid, pos):

    straight_down = pos + np.array([0, 1])
    straight_down_to_left = straight_down + np.array([-1, 0])
    straight_down_to_right = straight_down + np.array([1, 0])

    for test_pos in [straight_down, straight_down_to_left, straight_down_to_right]:
        if (np.array([0, 0]) > test_pos).any() or (grid.shape <= test_pos).any():
            raise AbyssError

    if grid[tuple(straight_down)] == ".":
        return sand_drop(grid, straight_down)
    elif grid[tuple(straight_down_to_left)] == ".":
        return sand_drop(grid, straight_down_to_left)
    elif grid[tuple(straight_down_to_right)] == ".":
        return sand_drop(grid, straight_down_to_right)
    elif grid[tuple(pos)] == "+":
        raise NozzleProblem
    else:
        grid[tuple(pos)] = "o"
        return pos

In [ ]:
rocks = [
    np.array([[int(c) for c in pos.split(",")] for pos in line.split(" -> ")])
    for line in data.split("\n")
]

min_x = min([min(path[:, 0]) for path in rocks])
rocks = [path - np.array([min_x, 0]) for path in rocks]
nozzle = 500 - min_x
grid = np.full(
    (get_edge(rocks, max, coord=0) + 1, get_edge(rocks, max, coord=1) + 1),
    ".",
    dtype="<U1",
)

map_out_rocks(rocks, grid)
grid[nozzle, 0] = "+"
viz_grid(grid)

for i in range(5000):
    try:
        sand_drop(grid, pos=np.array([nozzle, 0]))
    except AbyssError:
        break

print(i)

In [ ]:
rocks = [
    np.array([[int(c) for c in pos.split(",")] for pos in line.split(" -> ")])
    for line in data.split("\n")
]

max_y = get_edge(rocks, max, coord=1) + 2
min_x = 500 - (max_y)

rocks = [path - np.array([min_x, 0]) for path in rocks]

max_x = 2 * max_y
max_y = get_edge(rocks, max, coord=1)

rocks += [np.array([[0, max_y + 2], [max_x, max_y + 2]])]

grid = np.full(
    (get_edge(rocks, max, coord=0) + 1, get_edge(rocks, max, coord=1) + 1),
    ".",
    dtype="<U1",
)

nozzle = 500 - min_x

map_out_rocks(rocks, grid)
grid[nozzle, 0] = "+"

sand_count = 0

while True:
    sand_count += 1
    try:
        sand_drop(grid, pos=np.array([nozzle, 0]))
    except NozzleProblem:
        break

viz_grid(grid)
print(sand_count)